# For colab environment

In [1]:
# !pip install nltk transformers==4.35.0 torch==2.6.0 torchvision==0.21.0 datasets accelerate==0.24.0 huggingface==0.0.1 datasets==2.14.7

In [2]:
import torch 
print(torch.cuda.is_available())
print(torch.__version__)

True
2.6.0+cu124


In [3]:
# !git clone https://github.com/BernardMoy/NLP-PCL-Classification.git

In [4]:
# %cd NLP-PCL-Classification/

In [5]:
!nvidia-smi

Sat Feb 28 22:56:17 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX TITAN X     Off |   00000000:02:00.0  On |                  N/A |
| 25%   56C    P8             30W /  250W |     211MiB /  12288MiB |     25%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Load train and validation data set

In [6]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
from collections import Counter

df = pd.read_csv('data/dontpatronizeme_pcl.tsv', sep='\t')

# Remove rows with NA labels 
df = df.dropna() 

# Add a bool_labels column for binary classification
df["bool_labels"] = df["label"] > 1   # is PCL if >1

# train val split 
train_labels = pd.read_csv('data/train_semeval_parids-labels.csv')["par_id"]
val_labels = pd.read_csv('data/dev_semeval_parids-labels.csv')["par_id"]
df_train = df[df["par_id"].isin(train_labels)]
df_val = df[df["par_id"].isin(val_labels)]


# Tokenization

In [7]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoConfig, Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained("roberta-base") 

# Create text with contextual information 
def tokenize(df): 
    text_with_context = df["text"] # df["keyword"] + "</s><s>" + df["country_code"] + "</s><s>" + df["text"]

    encoding = tokenizer(
        text_with_context.tolist(), 
        padding="max_length",   # Add padding to shorter sentences 
        max_length=256,
        truncation = True, 
        return_attention_mask = True 
    )

    return encoding

/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


# Convert to pyTorch dataset

In [8]:
import torch 
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset

def to_dataset(df): 
    # Obtain tokens (input_ids, attention_mask) from the dataset 
    encoding = tokenize(df) 

    # Return huggingface dataset 
    return Dataset.from_dict({
        "input_ids": encoding["input_ids"], 
        "attention_mask": encoding["attention_mask"], 
        "label": df["bool_labels"].values 
    })

In [9]:
train_dataset = to_dataset(df_train)
val_dataset = to_dataset(df_val) 

# set to torch format 
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Training 

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate metrics 
    accuracy = accuracy_score(labels, predictions) 
    precision = precision_score(labels, predictions) 
    recall = recall_score(labels, predictions) 
    f1 = f1_score(labels, predictions) 

    return {
        "accuracy": accuracy, 
        "precision": precision, 
        "recall": recall, 
        "f1": f1 
    }


In [11]:
# Load roberta sequence classification model 
config = AutoConfig.from_pretrained("roberta-base", num_labels=2)  # Binary classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config = config)

BATCH_SIZE = 32
N_EPOCHS = 5 

# Set up training arguments 
training_args = TrainingArguments(
    fp16=True, 
    num_train_epochs=N_EPOCHS, 
    learning_rate=2e-5, 
    weight_decay=0.01,
    warmup_steps=500, 
    save_strategy="epoch",  # low disk space 
    load_best_model_at_end=True, 
    metric_for_best_model='f1',
    logging_steps=50,
    output_dir="./predictions", 
    evaluation_strategy="epoch", 
    per_device_eval_batch_size=BATCH_SIZE, 
    per_device_train_batch_size=BATCH_SIZE, 
)

# Set up trainer 
trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    compute_metrics=compute_metrics
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [12]:
trainer.train() 

  0%|          | 0/1310 [00:00<?, ?it/s]

{'loss': 0.677, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.19}
{'loss': 0.4275, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.38}
{'loss': 0.2987, 'learning_rate': 5.9600000000000005e-06, 'epoch': 0.57}
{'loss': 0.2383, 'learning_rate': 7.960000000000002e-06, 'epoch': 0.76}
{'loss': 0.2306, 'learning_rate': 9.960000000000001e-06, 'epoch': 0.95}


  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.21965700387954712, 'eval_accuracy': 0.9173435260391782, 'eval_precision': 0.6382978723404256, 'eval_recall': 0.3015075376884422, 'eval_f1': 0.40955631399317405, 'eval_runtime': 29.1853, 'eval_samples_per_second': 71.714, 'eval_steps_per_second': 2.261, 'epoch': 1.0}
{'loss': 0.2021, 'learning_rate': 1.196e-05, 'epoch': 1.15}
{'loss': 0.2225, 'learning_rate': 1.396e-05, 'epoch': 1.34}
{'loss': 0.2214, 'learning_rate': 1.5960000000000003e-05, 'epoch': 1.53}
{'loss': 0.1932, 'learning_rate': 1.796e-05, 'epoch': 1.72}
{'loss': 0.1904, 'learning_rate': 1.9960000000000002e-05, 'epoch': 1.91}


  0%|          | 0/66 [00:00<?, ?it/s]

...

  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.2779287099838257, 'eval_accuracy': 0.9273769708552317, 'eval_precision': 0.6715328467153284, 'eval_recall': 0.4623115577889447, 'eval_f1': 0.5476190476190477, 'eval_runtime': 29.2325, 'eval_samples_per_second': 71.598, 'eval_steps_per_second': 2.258, 'epoch': 4.0}
{'loss': 0.0783, 'learning_rate': 6.493827160493828e-06, 'epoch': 4.01}
{'loss': 0.026, 'learning_rate': 5.259259259259259e-06, 'epoch': 4.2}
{'loss': 0.0609, 'learning_rate': 4.024691358024692e-06, 'epoch': 4.39}
{'loss': 0.0237, 'learning_rate': 2.7901234567901235e-06, 'epoch': 4.58}
{'loss': 0.0477, 'learning_rate': 1.5555555555555558e-06, 'epoch': 4.77}
{'loss': 0.0449, 'learning_rate': 3.209876543209877e-07, 'epoch': 4.96}


  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.34054291248321533, 'eval_accuracy': 0.9283325370281892, 'eval_precision': 0.6416184971098265, 'eval_recall': 0.5577889447236181, 'eval_f1': 0.5967741935483871, 'eval_runtime': 29.1874, 'eval_samples_per_second': 71.709, 'eval_steps_per_second': 2.261, 'epoch': 5.0}
{'train_runtime': 1933.6341, 'train_samples_per_second': 21.656, 'train_steps_per_second': 0.677, 'train_loss': 0.16789695054633927, 'epoch': 5.0}


TrainOutput(global_step=1310, training_loss=0.16789695054633927, metrics={'train_runtime': 1933.6341, 'train_samples_per_second': 21.656, 'train_steps_per_second': 0.677, 'train_loss': 0.16789695054633927, 'epoch': 5.0})

In [13]:
trainer.evaluate()

  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.34054291248321533,
 'eval_accuracy': 0.9283325370281892,
 'eval_precision': 0.6416184971098265,
 'eval_recall': 0.5577889447236181,
 'eval_f1': 0.5967741935483871,
 'eval_runtime': 28.5979,
 'eval_samples_per_second': 73.187,
 'eval_steps_per_second': 2.308,
 'epoch': 5.0}

# Save model

In [22]:
trainer.save_model('baseline_model')

# Load model

In [23]:
import os 
# must pass abs path here 
model = RobertaForSequenceClassification.from_pretrained(os.path.abspath("/vol/bitbucket/bm1325/NLP-PCL-Classification/baseline_model"), local_files_only = True)

model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             